In [ ]:
import yaml
import os
from a_star import AStarTime
import numpy as np
from tqdm import tqdm
import time
from pprint import pprint

In [ ]:
import scipy.io as scio
def save_mat(path, to_save):
    '''
    Save data to mat file
    Args:
        path: path to save mat file
        to_save: data to save in the mat
    '''
    scio.savemat(path, to_save) 

# HCA Baseline Codes
Version: 1.0

Author: Weizhe Lin @ Feb 2021

In [ ]:
def hca(img, all_start, all_end, steps=100):
    '''
    HCA function
    Args:
        img: map
        all_start: start positions of agents
        all_end: goal positions of agents
        steps: maximum allowed steps
    Return:
        all_path: calculated paths of agents
    '''
    all_path = []
    robot_loc = np.where(img==3)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i,j] == 3:
                img[i,j] = 0        
    res_imgs = np.expand_dims(img, axis=0).repeat(steps, axis=0)
    for i in range(len(robot_loc[0])):
        res_imgs[0, robot_loc[0][i], robot_loc[1][i]] = 3
    for i in range(len(all_start)):
        robot_path = AStarTime(res_imgs, (all_start[i][0],all_start[i][1]), (all_end[i][0], all_end[i][1]))
        #print(i)
        if len(robot_path) == 1:
            new_path = []
            for j in range(steps-1):
                res_imgs[j, all_start[i][0],all_start[i][1]] = 3
                new_path.append([all_start[i][0],all_start[i][1],j])
            all_path.append(new_path)    
            continue
        else:
            for loc in robot_path:
                res_imgs[loc[2], loc[0], loc[1]] = 3
            all_path.append(robot_path)
    return all_path

In [ ]:
# Load dataset
DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map20x20_density_p1/10_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 100

In [ ]:
data = []
ecbs_data = []  # save expert(ECBS) data
for root, dirs, files in os.walk(os.path.join(DATA_FOLDER, 'input')):
    for f in files:
        print(len(data), end='\r')
        if len(data) >= DATA_NUM:
            break
        IDMAP = f.split('IDMap')[1].split('_')[0]
        IDCase = f.split('IDCase')[1].split('.yaml')[0]
        IDMAP = int(IDMAP)
        IDCase = int(IDCase)
        if IDMAP >= DATA_RANGE[0] and IDMAP <= DATA_RANGE[1]:
            with open(os.path.join(root, f),'r',encoding='utf-8') as fs:
                cont = fs.read()
                x = yaml.load(cont)
                
#             print(IDMAP, IDCase)
            
            output_name = f.replace('input', 'output').split('.yaml')[0] + '_ECBS.yaml'
            output_path = os.path.join(DATA_FOLDER, 'output_ECBS', output_name)
            if os.path.exists(output_path):
                with open(output_path ,'r',encoding='utf-8') as fs:
                    cont = fs.read()
                    y = yaml.load(cont)
#                     print(output_name)
                data.append(x)
                ecbs_data.append(y)
        
print(len(data))
print(len(ecbs_data))

In [ ]:
# flowtime_increase_list = []
# time_cost_list = []
# all_success_count = []
# individual_success_count = []
# dim_num = None

# for input_data, output_data in tqdm(zip(data, ecbs_data), total=len(data)):
#     success_count = []
#     start_time = time.time()
#     expert_makespan = output_data['statistics']['makespan']
#     expert_cost = output_data['statistics']['cost']
    
# #     print('===')
# #     print(input_data)
# #     print(output_data)
#     env = np.zeros(input_data['map']['dimensions'])
#     for obs in input_data['map']['obstacles']:
#         env[obs[0], obs[1]] = 1
#     if not dim_num:
#         dim_num = [input_data['map']['dimensions'][0], len(input_data['agents'])]
#     all_start = []
#     all_end = []
#     img = env.copy()
#     for agent_data in input_data['agents']:
#         start = agent_data['start']
#         end = agent_data['goal']
#         all_start.append(start)
#         all_end.append(end)
#         img[start[0], start[1]] = 3
        
#     all_paths = hca(img, all_start, all_end, steps=expert_makespan*3+1)

#     cost = 0
#     makespan = 0
#     collision_dict = {}
#     for agent_path, goal_pos in zip(all_paths, all_end):

#         final_x = agent_path[-1][0]
#         final_y = agent_path[-1][1]

#         if goal_pos[0] != final_x or goal_pos[1] != final_y:
#             # Did not reach end
#             cost += expert_makespan*3
#             success_count.append(0)
#         else:
#             # reach goal
#             cost += len(agent_path)
#             success_count.append(1)
#         makespan = max(makespan, len(agent_path))
        
#         for agent_path_t in agent_path:
#             t = agent_path_t[2]
#             x = agent_path_t[0]
#             y = agent_path_t[1]
#             pos = (x,y)
#             if pos in collision_dict.setdefault(t, {}).keys():
#                 print('found_collision!', t, collision_dict[t], agent_path)
#             else:
#                 collision_dict[t][pos] = True
    
#     end_time = time.time()
#     time_elapsed = end_time - start_time
#     time_cost_list.append(time_elapsed)
#     flowtime_increase = cost/expert_cost-1
#     flowtime_increase_list.append(flowtime_increase)
#     individual_success_count += success_count
# #     print((0 not in success_count), success_count)
# #     if (0 in success_count):
# #         print(input_data)
# #         print(output_data)
# #         print(all_paths[success_count.index(0)])
#     all_success_count.append((0 not in success_count))

In [ ]:
print('{}x{}({})'.format(dim_num[0], dim_num[0], dim_num[1]))
flowtime_increase_array = np.array(flowtime_increase_list)
time_cost_array = np.array(time_cost_list)
all_success_array = np.array(all_success_count)
individual_success_array = np.array(individual_success_count)
print('FT_increase;{};{}'.format(np.mean(flowtime_increase_array), np.std(flowtime_increase_array)))
print('time_cost;{};{}'.format(np.mean(time_cost_array), np.std(time_cost_array)))
print('all_success;{};{}'.format(np.mean(all_success_array), np.std(all_success_array)))
print('individual_success_rate;{};{}'.format(np.mean(individual_success_array), np.std(individual_success_array)))
print('{}x{}({})'.format(dim_num[0], dim_num[0], dim_num[1]),';','{};{};{};{};{};{};{};{}'.format(
        np.mean(flowtime_increase_array), np.std(flowtime_increase_array),
        np.mean(time_cost_array), np.std(time_cost_array),
        np.mean(all_success_array), np.std(all_success_array),
        np.mean(individual_success_array), np.std(individual_success_array),
    ))

In [ ]:
LOG_TIME = int(time.time())
DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map20x20_density_p1/10_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 10
run_in_pipeline()

In [ ]:
LOG_TIME = int(time.time())
DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map20x20_density_p1/10_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 4500
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map28x28_density_p1/20_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map35x35_density_p1/30_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map40x40_density_p1/40_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map45x45_density_p1/50_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map50x50_density_p1/60_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/EffectiveDensity/map65x65_density_p1/100_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

In [ ]:
LOG_TIME = 1612452524
DATA_FOLDER = '/media/pc/文档/Dataset/SameMap_diffRobot/map50x50_density_p1/10_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/SameMap_diffRobot/map50x50_density_p1/20_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/SameMap_diffRobot/map50x50_density_p1/30_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/SameMap_diffRobot/map50x50_density_p1/40_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/SameMap_diffRobot/map50x50_density_p1/50_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

DATA_FOLDER = '/media/pc/文档/Dataset/SameMap_diffRobot/map50x50_density_p1/100_Agent'
# DATA_FOLDER = '/home/pc/experiment_data/60_Agent'
DATA_RANGE = [427, 800]
DATA_NUM = 1000
run_in_pipeline()

In [ ]:
def run_in_pipeline():
    data = []
    ecbs_data = []
    for root, dirs, files in os.walk(os.path.join(DATA_FOLDER, 'input')):
        for f in files:
            print('loading...', len(data), end='\r')
            if len(data) >= DATA_NUM:
                break
            IDMAP = f.split('IDMap')[1].split('_')[0]
            IDCase = f.split('IDCase')[1].split('.yaml')[0]
            IDMAP = int(IDMAP)
            IDCase = int(IDCase)
            if IDMAP >= DATA_RANGE[0] and IDMAP <= DATA_RANGE[1]:
                with open(os.path.join(root, f),'r',encoding='utf-8') as fs:
                    cont = fs.read()
                    x = yaml.load(cont)

                output_name = f.replace('input', 'output').split('.yaml')[0] + '_ECBS.yaml'
                output_path = os.path.join(DATA_FOLDER, 'output_ECBS', output_name)
                if os.path.exists(output_path):
                    with open(output_path ,'r',encoding='utf-8') as fs:
                        cont = fs.read()
                        y = yaml.load(cont)
                    data.append(x)
                    ecbs_data.append(y)
          
    print('finished loading:', len(data))
    print(len(ecbs_data))
    
    flowtime_increase_list = []
    makespan_list = []
    time_cost_list = []
    all_success_count = []
    individual_success_count = []
    num_reachGoal_list = []
    dim_num = None


    for input_data, output_data in tqdm(zip(data, ecbs_data), total=len(data)):
        success_count = []
        start_time = time.time()
        expert_makespan = output_data['statistics']['makespan']
        expert_cost = output_data['statistics']['cost']

    #     print('===')
    #     print(input_data)
    #     print(output_data)
        env = np.zeros(input_data['map']['dimensions'])
        for obs in input_data['map']['obstacles']:
            env[obs[0], obs[1]] = 1
        if not dim_num:
            dim_num = [input_data['map']['dimensions'][0], len(input_data['agents'])]
        all_start = []
        all_end = []
        img = env.copy()
        for agent_data in input_data['agents']:
            start = agent_data['start']
            end = agent_data['goal']
            all_start.append(start)
            all_end.append(end)
            img[start[0], start[1]] = 3

        all_paths = hca(img, all_start, all_end, steps=expert_makespan*3+1)

        cost = 0
        makespan = 0
        collision_dict = {}
        for agent_path, goal_pos in zip(all_paths, all_end):

            final_x = agent_path[-1][0]
            final_y = agent_path[-1][1]

            if goal_pos[0] != final_x or goal_pos[1] != final_y:
                # Did not reach end
                cost += expert_makespan*3
                success_count.append(0)
            else:
                # reach goal
                cost += len(agent_path)
                success_count.append(1)
            makespan_list.append(len(agent_path))
            makespan = max(makespan, len(agent_path))

#             for agent_path_t in agent_path:
#                 t = agent_path_t[2]
#                 x = agent_path_t[0]
#                 y = agent_path_t[1]
#                 pos = (x,y)
#                 if pos in collision_dict.setdefault(t, {}).keys():
#                     print('found_collision!', t, collision_dict[t], agent_path)
#                 else:
#                     collision_dict[t][pos] = True

        end_time = time.time()
        time_elapsed = end_time - start_time
        time_cost_list.append(time_elapsed)
        flowtime_increase = cost/expert_cost-1
        flowtime_increase_list.append(flowtime_increase)
        individual_success_count += success_count
        all_success_count.append((0 not in success_count))
        
        num_reachGoal_list.append(np.count_nonzero(np.array(success_count)))
        
    print('{}x{}({})'.format(dim_num[0], dim_num[0], dim_num[1]))
    flowtime_increase_array = np.array(flowtime_increase_list)
    makespan_array = np.array(makespan_list)
    time_cost_array = np.array(time_cost_list)
    all_success_array = np.array(all_success_count)
    individual_success_array = np.array(individual_success_count)
    print('FT_increase;{};{}'.format(np.mean(flowtime_increase_array), np.std(flowtime_increase_array)))
    print('time_cost;{};{}'.format(np.mean(time_cost_array), np.std(time_cost_array)))
    print('all_success;{};{}'.format(np.mean(all_success_array), np.std(all_success_array)))
    print('individual_success_rate;{};{}'.format(np.mean(individual_success_array), np.std(individual_success_array)))
    
    log_time = LOG_TIME
    mat_data = {
        'rate_ReachGoal':[[np.mean(all_success_array)]],
        'num_agents_trained':[[dim_num[1]]],
        'num_agents_testing':[[dim_num[1]]],
        'map_size_testing':[[dim_num[0], dim_num[0]]],
        'map_size_trained': [[dim_num[0], dim_num[0]]],
        'map_density_trained': [[0.1]],
        'map_density_testing': [[0.1]],
        'K':[[0]],
        'trained_model_epoch':[[0]],
        'log_time':[[log_time]],
        'std_deltaMP': [[np.std(makespan_array)]],
        'mean_deltaMP':[[np.mean(makespan_array)]],
        'list_deltaMP':[[makespan_list]],
        'mean_deltaFT':[[np.mean(flowtime_increase_array)]],
        'std_deltaFT':[[np.std(flowtime_increase_array)]],
        'list_deltaFT': [flowtime_increase_list],
        'list_reachGoal':all_success_count,
        'list_computationTime':[time_cost_list],
        'list_numAgentReachGoal':[num_reachGoal_list],
        'action_policy': 'exp_multinorm',
        'hidden_state': [[0]],
    }
    pprint(mat_data)
    file_name = 'HCA_{}x{}({})_{}_exp_multinorm.mat'.format(dim_num[0], dim_num[0], dim_num[1], log_time)
    save_mat(file_name, mat_data)
    print('{}x{}({})'.format(dim_num[0], dim_num[0], dim_num[1]),';','{};{};{};{};{};{};{};{}'.format(
        np.mean(flowtime_increase_array), np.std(flowtime_increase_array),
        np.mean(time_cost_array), np.std(time_cost_array),
        np.mean(all_success_array), np.std(all_success_array),
        np.mean(individual_success_array), np.std(individual_success_array),
    ))
    
    print('========done========')